In [2]:
###installing dependencies
import pandas as pd
import numpy as np
import os
import glob
import yfinance as yf
import datetime
import re
import emoji
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob


In [30]:
os.chdir("/Users/pranavtavildar/Desktop/Senior-Thesis/data/monthlydata")  
#creating big csv
extension = 'csv'
all_filenames = sorted([i for i in glob.glob('*.{}'.format(extension))])
#combine all files in the list
#all_filenames.remove('CVX-Tweets-12-2022-310.csv')


combined_csv = pd.DataFrame()
i = 0
for f in all_filenames:
    try:
        df = pd.read_csv(f, encoding='utf-8-sig',lineterminator='\n')
        # df.info()
        combined_csv = pd.concat([combined_csv, df], ignore_index=True, verify_integrity=True)
        i += 1
    except Exception as e:
        print(i, "files were read and combined properly")
        # df.info()
        print(df.describe())
        print(f"Error reading file {f}: {str(e)}")
        

# #export to csv
combined_csv.to_csv( "../CVXTweets.csv", index=False, encoding='utf-8-sig',lineterminator='\n')

In [31]:
os.chdir("/Users/pranavtavildar/Desktop/Senior-Thesis/data")

# Retrieve the data from Yahoo Finance
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2022, 12, 31)

cvx_data = yf.download('CVX', start=start_date, end=end_date)

cvx_data.to_csv('CVX_stock_data.csv')



[*********************100%***********************]  1 of 1 completed


In [32]:
# Loading stock data 
stock_data = pd.read_csv("CVX_stock_data.csv")
stock_data["Date"] = pd.to_datetime(stock_data["Date"])
stock_data = stock_data.set_index("Date")

# Loading tweets
tweets = pd.read_csv("CVXTweets.csv",encoding='utf-8-sig',lineterminator='\n')
tweets["Date"] = pd.to_datetime(tweets["Date"])

# Filtering tweets
filtered_tweets = tweets.loc[tweets["Date"].dt.date.isin(stock_data.index.date)]

# Save the filtered tweets to a new CSV file
filtered_tweets.to_csv("CVXTweets_filtered.csv", index=False)


In [33]:
###Preprocessing Tweets
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    r = tweet.lower()
    r = re.sub("'", "", r) 
    r = re.sub("@[A-Za-z0-9_]+","", r)
    r = re.sub("#[A-Za-z0-9_]+","", r)
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r

# Apply the clean_tweet function 
filtered_tweets['cleaned_text'] = filtered_tweets['Text'].apply(clean_tweet)

filtered_tweets.to_csv('CVXTweets_cleaned.csv', index=False)

unfiltered_tweets = pd.read_csv("CVXTweets_filtered.csv",encoding='utf-8-sig',lineterminator='\n')

print(unfiltered_tweets.equals(filtered_tweets))
unfiltered_tweets.head()

False


/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_670/2702314473.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(tweet) == np.float:
/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_670/2702314473.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tweets['cleaned_text'] = filtered_tweets['Text'].apply(clean_tweet)


,Date,Tweet ID,Text,Likes
0,2021-01-04 23:47:11+00:00,1346241797039656960,Chevron is my way of getting a high dividend y...,0
1,2021-01-04 22:51:14+00:00,1346227718539649024,Chevron $CVX announced today they'd be holding...,4
2,2021-01-04 22:38:50+00:00,1346224598740819969,$CVX Advisory: Chevron Corporation’s 4Q 2020 E...,0
3,2021-01-04 22:32:53+00:00,1346223100879364102,Advisory: Chevron’s 4Q 2020 Earnings Conferenc...,0
4,2021-01-04 21:57:45+00:00,1346214260553248769,Electric vehicle charging network ChargePoint ...,0


In [34]:
###Sentiment Analysis 

filtered_tweets['sentiment'] = filtered_tweets['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

filtered_tweets.to_csv('CVXTweets_sentiments.csv', index=False)
filtered_tweets.head()


/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_670/1803340424.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tweets['sentiment'] = filtered_tweets['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)


,Date,Tweet ID,Text,Likes,cleaned_text,sentiment
30,2021-01-04 23:47:11+00:00,1346241797039656960,Chevron is my way of getting a high dividend y...,0,chevron is my way getting high dividend yield ...,0.015
31,2021-01-04 22:51:14+00:00,1346227718539649024,Chevron $CVX announced today they'd be holding...,4,chevron cvx announced today theyd be holding t...,0.000
32,2021-01-04 22:38:50+00:00,1346224598740819969,$CVX Advisory: Chevron Corporation’s 4Q 2020 E...,0,cvx advisory chevron corporation s 4q 2020 ear...,0.000
33,2021-01-04 22:32:53+00:00,1346223100879364102,Advisory: Chevron’s 4Q 2020 Earnings Conferenc...,0,advisory chevron s 4q 2020 earnings conference...,0.000
34,2021-01-04 21:57:45+00:00,1346214260553248769,Electric vehicle charging network ChargePoint ...,0,electric vehicle charging network chargepoint ...,0.000


In [54]:
##Debug from here
# group the data by Date and calculate the mean sentiment
os.chdir("/Users/pranavtavildar/Desktop/Senior-Thesis/data")  
filtered_tweets = pd.read_csv("CVXTweets_sentiments.csv",encoding='utf-8-sig',lineterminator='\n')
filtered_tweets['Date'] = pd.to_datetime(filtered_tweets['Date']).dt.date
avg_sentiment = filtered_tweets.groupby(['Date'])['sentiment'].mean().reset_index()
avg_sentiment.to_csv('daily_sentiments.csv', index=False)


In [56]:
#adjusting the weighted sentiment score so that the baseline is 1
filtered_tweets['adjusted_likes'] = filtered_tweets['Likes']+1
print(min(filtered_tweets['adjusted_likes']))

# calculate the weighted sentiment by multiplying the sentiment by Likes
filtered_tweets['weighted_sentiment'] = filtered_tweets['sentiment'] * filtered_tweets['adjusted_likes']

# group the data by Date and calculate the weighted mean sentiment
weighted_avg_sentiment = filtered_tweets.groupby(['Date'])['weighted_sentiment', 'adjusted_likes'].sum().reset_index()
weighted_avg_sentiment['weighted_sentiment'] /= weighted_avg_sentiment['adjusted_likes']

weighted_avg_sentiment.to_csv('weighted_daily_sentiments.csv', index=False)
weighted_avg_sentiment['weighted_sentiment'].head()
# missing_dates = weighted_avg_sentiment.loc[weighted_avg_sentiment['weighted_sentiment'].isna()].index
# print(missing_dates)
# missing_tweets = filtered_tweets[filtered_tweets['Date'].isin(missing_dates)]

1


/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_670/812023121.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weighted_avg_sentiment = filtered_tweets.groupby(['Date'])['weighted_sentiment', 'adjusted_likes'].sum().reset_index()


0    0.184861
1    0.371769
2    0.220826
3    0.032773
4    0.008593
Name: weighted_sentiment, dtype: float64